# Chebyshev Pseudospectral Method Example

### Toy model

This notebook follows the introductory example from 
- Olver, Townsend, A fast and well-conditioned spectral method, arXiv:1202.1347

We consider the first-order boundary value problem on the interval $[-1, 1]$. 
$$
  u'(x) + x u(x) = f(x), \qquad x(-1) = 0.
$$

In [ ]:
using OrdinaryDiffEq, Plots

f_fun = x -> cos(3*sin(3*x))
F(u, p, x) = - x * u + f_fun(x)
u0 = 0.0 
xspan = (-1.0, 1.0)
prob = ODEProblem(F, u0, xspan)
sol = solve(prob, Tsit5(), reltol = 1e-8, abstol = 1e-8)

p1 = plot(sol, lw = 3, xaxis = "x", yaxis = "u(x)", 
          label = "ODE-Solver", size = (400, 250))

In [ ]:
using LinearAlgebra, FFTW, BandedMatrices, Plots

function chebbasis(x, N)
   T = zeros(N+1)
   T[1] = 1 
   T[2] = x 
   for n = 2:N
      T[n+1] = 2 * x * T[n] - T[n-1] 
   end 
   return T 
end

chebnodes(N) = [ cos( π * n / N ) for n = N:-1:0 ]

function fct(A::AbstractVector)
    N = length(A)
    F = real.(ifft([A[1:N]; A[N-1:-1:2]]))
   return [[F[1]]; 2*F[2:(N-1)]; [F[N]]]
end

"""
Fast and stable implementation based on the FFT. This uses 
the connection between Chebyshev and trigonometric interpolation.
But this transform needs the reverse chebyshev nodes.
"""
chebinterp(f, N) = fct(f.(reverse(chebnodes(N))))

"""
Evaluate a polynomial with coefficients F̃ in the Chebyshev basis. 
This avoids storing the basis and is significantly faster.
"""
function chebeval(x, F̃) 
    T0 = one(x); T1 = x 
    p = F̃[1] * T0 + F̃[2] * T1 
    for n = 3:length(F̃)
        T0, T1 = T1, 2*x*T1 - T0 
        p += F̃[n] * T1 
    end 
    return p 
end 


In [ ]:
# The multiplication operator T ↦ x * T
𝒳 = N -> BandedMatrix(( -1 => fill(0.5, N), 
                         1 => [[1.0]; fill(0.5, N-1)]), (N+1, N+1))
𝒳(8)

In [ ]:
# The differentiation operator T ↦ T' = 𝐃 * U
𝒟 = N -> BandedMatrix(( -1 => Float64.(1:N),), (N+1, N+1) )
𝒟(8)

In [ ]:
# The basis transform T = 𝒮 * U
𝒮 = N -> BandedMatrix( (-2 => fill(-0.5, N-1), 
                         0 => [[1.0]; fill(0.5, N)]), (N+1, N+1) )
𝒮(8)

With the operators defined above the chebyshev pseudo-spectral method becomes 
$$
  (\mathscr{D}^T + \mathscr{S}^T \mathscr{X}^T) {\bf c} = \mathscr{S}^T {\bf F}.
$$
(except we haven't yet applied the boundary condition!)

In [ ]:
𝒜 = N -> 𝒟(N)' + 𝒮(N)' * 𝒳(N)'
𝒜(8)

In [ ]:
# Since we are missing the boundary condition, it is reasonable 
# to assume that this is not invertible! Indeed we can see this in 
# the spectrum.
eigvals(collect(𝒜(8)))

In [ ]:
N = 8
[ chebbasis(-1.0, N)'; 𝒜(N) ]

In [ ]:
# We do something really naive now to solve the 
# resulting linear system, there is a lot one can 
# do to make this efficient (see cited reference)
# This would need to exploit the fact that we 
# can decompose 𝐀 into a banded and a low-rank matrix.

N = 12
𝐀 = [ chebbasis(-1.0, N)'; 𝒜(N) ]
𝐅 = chebinterp(f_fun, N)
𝐆 = [ [0.0]; 𝒮(N)' * 𝐅 ]
𝐜 = 𝐀 \ 𝐆;  # treat as LSQ, uses qr factorization

xp = range(-1.0, 1.0, length=300)
up = chebeval.(xp, Ref(𝐜))

plot!(deepcopy(p1), xp, up, lw=3, label = "Chebyshev Method")